In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
 
#from google.colab import drive
#drive.mount('/content/drive')
 
datapath='/content/drive/MyDrive/'
model = tf.keras.models.load_model(datapath)
print("model loaded")

model loaded


In [8]:
from IPython.display import display, Javascript
import IPython
from google.colab.output import eval_js
from google.colab import output
from PIL import Image
from skimage import transform
from base64 import b64decode
import time
from io import BytesIO

htm='''<div id="output">
  Result: <div id="result"></div><br>
  <video width="300" height="300" id="vdo" style="display:block"></video>
</div>

<script>
  var video,stream;
  (async function a(){
    video = document.getElementById("vdo");
    stream = await navigator.mediaDevices.getUserMedia({video: true});
    video.srcObject = stream;
    video.play();
  })();

  async function takePhoto(quality) {
    var canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext('2d').drawImage(video, 0, 0);
    // stream.getVideoTracks()[0].stop();
    var data= canvas.toDataURL('image/jpeg', quality)

    const result = await google.colab.kernel.invokeFunction(
     'notebook.Predict', // The callback name.
      [data], // The arguments.
      {}); // kwargs
    const text = result.data['application/json'];
    document.querySelector("#result").innerHTML=text.result;
    setTimeout(function(){takePhoto(quality)},100);
  }


  takePhoto(0.8);
</script>
'''

def Predict(data):
  try:

    LABELS=['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic', 'Trash']
  #  binary = b64decode(data.split(',')[1])
  #  filename="test.jpg"
  #  with open(filename, 'wb') as f:
  #    f.write(binary)
  #  image = load(filename)
    np_image = Image.open(BytesIO(b64decode(data.split(',')[1])))
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (299, 299, 3))
    np_image = np.expand_dims(np_image, axis=0)
    a=model.predict(np_image)
    b=np.argmax(a,axis=1)[0]
    return IPython.display.JSON({'result': LABELS[b]+"<br><img src='"+data+"'>"})
  except Exception as err:
    return IPython.display.JSON({'result': '[ERROR]'+str(err)})

output.register_callback('notebook.Predict', Predict)

IPython.display.HTML(htm)
